In [2]:
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from dotenv import load_dotenv
import os

load_dotenv()


connection_parameters = {
   "account": os.getenv('account_snow'),
   "user": os.getenv('user_snow'),
   "password":os.getenv('password_snow'),
   "role": "ACCOUNTADMIN",
   "database": "NEW_DB", 
   "schema": "PUBLIC"
  
}  

session = Session.builder.configs(connection_parameters).create()

In [3]:
## Creation of Price table
session.sql("create or replace table prices(Product_id varchar,\
                Amount number(10,2),\
                Inserted_Timestamp Timestamp)").collect()

[Row(status='Table PRICES successfully created.')]

In [4]:
session.sql("insert into prices values \
              ('id1', 10.0, CURRENT_TIMESTAMP()), \
              ('id2', 20.0, CURRENT_TIMESTAMP()), \
              ('id3', 30.0, CURRENT_TIMESTAMP()), \
              ('id4', 40.0, CURRENT_TIMESTAMP()), \
              ('id5', 50.0, CURRENT_TIMESTAMP()), \
              ('id6', 60.0, CURRENT_TIMESTAMP()), \
              ('id7', 70.0, CURRENT_TIMESTAMP()), \
              ('id8', 80.0, CURRENT_TIMESTAMP()), \
              ('id9', 90.0, CURRENT_TIMESTAMP()), \
              ('id10', 100.0, CURRENT_TIMESTAMP()), \
              ('id11', 110.0, CURRENT_TIMESTAMP())").collect()

[Row(number of rows inserted=11)]

In [5]:
session.sql("SELECT count(*) FROM prices").collect()

[Row(COUNT(*)=11)]

In [6]:
df_table = session.table("prices")

In [7]:
df_table.show()

--------------------------------------------------------
|"PRODUCT_ID"  |"AMOUNT"  |"INSERTED_TIMESTAMP"        |
--------------------------------------------------------
|id1           |10.00     |2025-02-01 23:19:25.001000  |
|id2           |20.00     |2025-02-01 23:19:25.001000  |
|id3           |30.00     |2025-02-01 23:19:25.001000  |
|id4           |40.00     |2025-02-01 23:19:25.001000  |
|id5           |50.00     |2025-02-01 23:19:25.001000  |
|id6           |60.00     |2025-02-01 23:19:25.001000  |
|id7           |70.00     |2025-02-01 23:19:25.001000  |
|id8           |80.00     |2025-02-01 23:19:25.001000  |
|id9           |90.00     |2025-02-01 23:19:25.001000  |
|id10          |100.00    |2025-02-01 23:19:25.001000  |
--------------------------------------------------------



In [8]:
from snowflake.snowpark.functions import asc, desc, col

# Sort the "prices" table by the "price" column in ascending order.
df_sort = session.table("prices").sort(asc(col("Amount")))
df_sort.show()

--------------------------------------------------------
|"PRODUCT_ID"  |"AMOUNT"  |"INSERTED_TIMESTAMP"        |
--------------------------------------------------------
|id1           |10.00     |2025-02-01 23:19:25.001000  |
|id2           |20.00     |2025-02-01 23:19:25.001000  |
|id3           |30.00     |2025-02-01 23:19:25.001000  |
|id4           |40.00     |2025-02-01 23:19:25.001000  |
|id5           |50.00     |2025-02-01 23:19:25.001000  |
|id6           |60.00     |2025-02-01 23:19:25.001000  |
|id7           |70.00     |2025-02-01 23:19:25.001000  |
|id8           |80.00     |2025-02-01 23:19:25.001000  |
|id9           |90.00     |2025-02-01 23:19:25.001000  |
|id10          |100.00    |2025-02-01 23:19:25.001000  |
--------------------------------------------------------



In [9]:
from snowflake.snowpark.functions import col

# Create a DataFrame for the rows with the ID 1
# in the "sample_product_data" table.

df_filter = session.table("prices").filter(col("product_id") == 'id1')
df_filter.show()

--------------------------------------------------------
|"PRODUCT_ID"  |"AMOUNT"  |"INSERTED_TIMESTAMP"        |
--------------------------------------------------------
|id1           |10.00     |2025-02-01 23:19:25.001000  |
--------------------------------------------------------



In [18]:
## Adding new records to the Dataframe

# Import the lit function from the functions module.
from snowflake.snowpark.functions import lit

# Create a new DataFrame with the values for the new row.
new_row = session.createDataFrame([( "Id1", 34, '2023-03-28 10:09:50.477000')], ['product_id', 'amount', 'inserted_timestamp'])

# Create a DataFrame for the existing rows in the "prices" table.
existing_rows = session.table("prices")

# Combine the two DataFrames into a single DataFrame.
df_combined_rows = existing_rows.union(new_row)

# Show the resulting DataFrame.
df_combined_rows.show(15)

--------------------------------------------------------
|"PRODUCT_ID"  |"AMOUNT"  |"INSERTED_TIMESTAMP"        |
--------------------------------------------------------
|id3           |30.00     |2025-02-01 23:19:25.001000  |
|id4           |40.00     |2025-02-01 23:19:25.001000  |
|id5           |50.00     |2025-02-01 23:19:25.001000  |
|id7           |70.00     |2025-02-01 23:19:25.001000  |
|id9           |90.00     |2025-02-01 23:19:25.001000  |
|id10          |100.00    |2025-02-01 23:19:25.001000  |
|id6           |60.00     |2025-02-01 23:19:25.001000  |
|Id1           |34.00     |2023-03-28 10:09:50.477000  |
|id8           |80.00     |2025-02-01 23:19:25.001000  |
|id11          |110.00    |2025-02-01 23:19:25.001000  |
--------------------------------------------------------



In [19]:
# Import the sum function from the functions module.
from snowflake.snowpark.functions import sum

# Calculate the total price for each product in the "prices" table.
df_aggregate = session.table("prices").groupBy(col("product_id"))\
                      .agg(sum(col("Amount")).alias("total_price"))\
                      .filter(col("product_id") == 'Id1')
df_aggregate.show(15)

--------------------------------
|"PRODUCT_ID"  |"TOTAL_PRICE"  |
--------------------------------
|              |               |
--------------------------------



In [11]:
# Use session.sql to execute a SQL delete statement to remove records with product_id 'id1' from the prices table
session.sql("DELETE FROM prices WHERE product_id = 'id1'").collect()

# Retrieve the updated table and assign it to a dataframe object
df = session.table("prices")

# Display the contents of the dataframe to verify that the appropriate records have been deleted
df.show()

--------------------------------------------------------
|"PRODUCT_ID"  |"AMOUNT"  |"INSERTED_TIMESTAMP"        |
--------------------------------------------------------
|id2           |20.00     |2025-02-01 23:19:25.001000  |
|id3           |30.00     |2025-02-01 23:19:25.001000  |
|id4           |40.00     |2025-02-01 23:19:25.001000  |
|id5           |50.00     |2025-02-01 23:19:25.001000  |
|id6           |60.00     |2025-02-01 23:19:25.001000  |
|id7           |70.00     |2025-02-01 23:19:25.001000  |
|id8           |80.00     |2025-02-01 23:19:25.001000  |
|id9           |90.00     |2025-02-01 23:19:25.001000  |
|id10          |100.00    |2025-02-01 23:19:25.001000  |
|id11          |110.00    |2025-02-01 23:19:25.001000  |
--------------------------------------------------------



In [12]:
# Import the necessary Snowpark functions
from snowflake.snowpark.functions import ltrim, rtrim

# Retrieve the existing table and assign it to a DataFrame object
df = session.table("prices")

# Create a new DataFrame without the records to delete using the `filter` method
filteredDf = df.filter(ltrim(rtrim(col("PRODUCT_ID"))) != 'id2')

# Display the contents of the new DataFrame
print("Filtered Temp Dataframe")
filteredDf.show()

# Write the filtered DataFrame to a temporary table
filteredDf.write.save_as_table("temp_table", mode="append", table_type="temporary")

# Drop the original table and recreate it with the contents of the temporary table
session.sql("DROP TABLE prices").collect()
session.sql("CREATE TABLE prices AS SELECT * FROM temp_table").collect()

# Retrieve the updated table and display its contents
df_Final = session.table("prices")
print("Final Table Value")
df_Final.show()

Filtered Temp Dataframe
--------------------------------------------------------
|"PRODUCT_ID"  |"AMOUNT"  |"INSERTED_TIMESTAMP"        |
--------------------------------------------------------
|id3           |30.00     |2025-02-01 23:19:25.001000  |
|id4           |40.00     |2025-02-01 23:19:25.001000  |
|id5           |50.00     |2025-02-01 23:19:25.001000  |
|id6           |60.00     |2025-02-01 23:19:25.001000  |
|id7           |70.00     |2025-02-01 23:19:25.001000  |
|id8           |80.00     |2025-02-01 23:19:25.001000  |
|id9           |90.00     |2025-02-01 23:19:25.001000  |
|id10          |100.00    |2025-02-01 23:19:25.001000  |
|id11          |110.00    |2025-02-01 23:19:25.001000  |
--------------------------------------------------------

Final Table Value
--------------------------------------------------------
|"PRODUCT_ID"  |"AMOUNT"  |"INSERTED_TIMESTAMP"        |
--------------------------------------------------------
|id3           |30.00     |2025-02-01 23:19:2

In [13]:
# Write data to the table in "append" mode
df_Final.write.mode("append").saveAsTable("Test_Demo_Append")

In [14]:
df_Final.write.mode("overwrite").saveAsTable("Test_Demo_Append")

In [15]:
## Write data to a table with "ignore" mode
df_Final.write.mode("ignore").saveAsTable("Test_Demo_Append")

In [16]:
# This mode throws an error if the table already exists.
# This is the default mode in Snowpark DataFrame.

df_Final.write.mode("errorifexists").saveAsTable("Test_Demo_Append")

SnowparkSQLException: (1304): 01ba1e69-0000-c7b4-0000-7d5d0004001a: 002002 (42710): SQL compilation error:
Object 'TEST_DEMO_APPEND' already exists.